```{contents}
```
## Execution Thread 

An **Execution Thread** in LangGraph is a **persistent, stateful control channel** that manages the lifecycle of a single graph run.
It represents **one continuous workflow instance** — from entry node to termination — and preserves identity, memory, and checkpoints across time.

---

### **1. Conceptual Definition**

An execution thread is:

> A **unique, long-lived execution context** that carries state, memory, checkpoints, and control flow for one invocation of a LangGraph application.

It enables:

* Stateful workflows
* Multi-turn conversations
* Human-in-the-loop execution
* Pausing, resuming, and recovery

---

### **2. Why Execution Threads Exist**

Without threads, LLM workflows are **stateless function calls**.

With threads, workflows become **persistent programs**.

| Capability     | Stateless Call    | Execution Thread |
| -------------- | ----------------- | ---------------- |
| Memory         | ❌ Lost after call | ✅ Persistent     |
| Pause / Resume | ❌                 | ✅                |
| Multi-turn     | ❌                 | ✅                |
| Checkpointing  | ❌                 | ✅                |
| Recovery       | ❌                 | ✅                |

---

### **3. Execution Thread Components**

Each execution thread encapsulates:

| Component          | Role                       |
| ------------------ | -------------------------- |
| Thread ID          | Unique identity of the run |
| State              | Shared mutable data        |
| Checkpoint History | Saved snapshots            |
| Execution Cursor   | Current node pointer       |
| Execution Log      | Full trace                 |
| Memory             | Short & long-term memory   |
| Pending Tasks      | In-flight operations       |
| Timeouts           | Safety controls            |

---

### **4. Execution Thread Lifecycle**

```
Create → Initialize → Execute → Checkpoint → Pause/Resume → Terminate → Archive
```

---

### **5. Creating an Execution Thread**

Threads are created implicitly via invocation:

```python
result = graph.invoke(input, config={"thread_id": "user-123"})
```

If the thread exists → resumes
If not → creates new thread

---

### **6. Threaded Multi-Turn Example**

```python
graph.invoke({"message": "Plan my trip"}, config={"thread_id": "alice"})
graph.invoke({"message": "Book hotel"}, config={"thread_id": "alice"})
```

Both calls operate on the **same execution thread** and **same state**.

---

### **7. Checkpointing & Recovery**

Every node execution creates a checkpoint:

```
[Node A] → Checkpoint 1
[Node B] → Checkpoint 2
[Node C] → Checkpoint 3
```

If failure occurs:

```python
graph.invoke(new_input, config={"thread_id": "alice"})
```

Execution resumes from the last safe checkpoint.

---

### **8. Thread Safety & Concurrency**

| Feature          | Behavior                         |
| ---------------- | -------------------------------- |
| Thread Isolation | Threads do not share state       |
| Concurrency      | Multiple threads run in parallel |
| Consistency      | One active cursor per thread     |
| Locking          | Prevents conflicting updates     |

---

### **9. Human-in-the-Loop via Threads**

```python
graph.invoke(input, config={"thread_id": "case-42", "interrupt_after": ["review"]})
```

Execution pauses at `review`.
Human modifies state → thread resumes.

---

### **10. Production Use Cases**

| Use Case   | Role of Thread           |
| ---------- | ------------------------ |
| Chatbots   | Each user = one thread   |
| Agents     | One task = one thread    |
| Workflows  | One job = one thread     |
| Auditing   | One trace per thread     |
| Compliance | Immutable thread history |

---

### **11. Mental Model**

An execution thread is to LangGraph what:

* a **process** is to an OS
* a **transaction** is to a database
* a **session** is to a web app

It transforms LLM workflows from **functions** into **programs**.

---

**12. Summary**

> **Execution threads give LangGraph memory, continuity, safety, and control.**
> They are the backbone of **reliable, long-running, production-grade LLM systems**.



In [37]:
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
from typing import TypedDict

class State(TypedDict):
    messages: list

def chatbot(state: State):
    new_msg = f"Bot: I received -> {state['messages'][-1]}"
    return {"messages": state["messages"] + [new_msg]}



In [38]:
builder = StateGraph(State)

builder.add_node("chat", chatbot)
builder.set_entry_point("chat")
builder.add_edge("chat", END)

memory = MemorySaver()
graph = builder.compile(checkpointer=memory)


In [39]:
config = {"configurable": {"thread_id": "user-1"}}

out1 = graph.invoke({"messages": ["Hello"]}, config)
print(out1)


{'messages': ['Hello', 'Bot: I received -> Hello']}


In [40]:
out2 = graph.invoke({"messages": ["How are you?"]}, config)
print(out2)


{'messages': ['How are you?', 'Bot: I received -> How are you?']}
